In [34]:
import requests
from bs4 import BeautifulSoup
import json
import pandas

In [23]:
import requests
from bs4 import BeautifulSoup

# Make a request to the website
url = "https://www.nba.com/game/bos-vs-dal-0042300404/play-by-play?latest=0&period=All"
response = requests.get(url)

# Parse the HTML content
soup = BeautifulSoup(response.content, "html.parser")

plays = soup.find('script', {"id": '__NEXT_DATA__'})
d = json.loads(plays.string)
play_by_play = d['props']['pageProps']['playByPlay']['actions']

In [109]:
def get_game_links(date):
    # date: string formatted as yyyy-mm-dd
    links = []
    url = f"https://www.nba.com/games?date={date}"
    response = requests.get(url)

    # Parse HTML with bs4
    soup = BeautifulSoup(response.content, "html.parser")
    data = soup.find('script', {"id": '__NEXT_DATA__'})
    d = json.loads(data.string) # bigass json stored as python dict 
    modules = d['props']['pageProps']['gameCardFeed']['modules']
    if modules:
        cards = d['props']['pageProps']['gameCardFeed']['modules'][0]['cards']
        for card in cards:
            url_chunk = card['cardData']['actions'][-1]['resourceLocator']['resourceUrl']
            links.append('https://www.nba.com' + url_chunk)
        return links
    else:
        return

In [129]:
def parse_game(ref_foul_data, link):

    url = link + 'play-by-play?latest=0&period=All'
    print(url)

    response = requests.get(url)

    # Parse HTML with bs4
    soup = BeautifulSoup(response.content, "html.parser")
    plays = soup.find('script', {"id": '__NEXT_DATA__'})
    d = json.loads(plays.string)
    play_by_play = d['props']['pageProps']['playByPlay']['actions']

    for play in play_by_play:
        if play['actionType'] == 'Foul':
            ref = play['description'].split()[-1]
            foul_type = play['subType']

            print(ref, foul_type)

            if ref not in ref_foul_data:
                ref_foul_data[ref] = {}
            

            print(ref_foul_data[ref])
            if foul_type not in ref_foul_data[ref]:
                ref_foul_data[ref][foul_type] = 0
            
            ref_foul_data[ref][foul_type] += 1

In [112]:
april_ninth_links = get_game_links('2024-04-09')

In [117]:
april_ninth_links

['https://www.nba.com/game/ind-vs-tor-0022301146',
 'https://www.nba.com/game/det-vs-phi-0022301145',
 'https://www.nba.com/game/dal-vs-cha-0022301144',
 'https://www.nba.com/game/bos-vs-mil-0022301148',
 'https://www.nba.com/game/mia-vs-atl-0022301147',
 'https://www.nba.com/game/sac-vs-okc-0022301153',
 'https://www.nba.com/game/was-vs-min-0022301152',
 'https://www.nba.com/game/sas-vs-mem-0022301151',
 'https://www.nba.com/game/orl-vs-hou-0022301150',
 'https://www.nba.com/game/nyk-vs-chi-0022301149',
 'https://www.nba.com/game/den-vs-uta-0022301154',
 'https://www.nba.com/game/nop-vs-por-0022301157',
 'https://www.nba.com/game/lac-vs-phx-0022301156',
 'https://www.nba.com/game/gsw-vs-lal-0022301155']

In [130]:
ref_foul_data = {}
for link in april_ninth_links:
    parse_game(ref_foul_data=ref_foul_data,
               link=link)

ref_foul_data

https://www.nba.com/game/ind-vs-tor-0022301146play-by-play?latest=0&period=All
(D.Taylor) Loose Ball
{}
(D.Taylor) Shooting
{'Loose Ball': 1}
(D.Taylor) Shooting
{'Loose Ball': 1, 'Shooting': 1}
(D.Taylor) Shooting
{'Loose Ball': 1, 'Shooting': 2}
(D.Taylor) Shooting
{'Loose Ball': 1, 'Shooting': 3}
(C.Kirkland) Shooting
{}
(D.Mosher) Personal
{}
(D.Mosher) Shooting
{'Personal': 1}
(D.Taylor) Shooting
{'Loose Ball': 1, 'Shooting': 4}
(C.Kirkland) Shooting
{'Shooting': 1}
(C.Kirkland) Technical
{'Shooting': 2}
(C.Kirkland) Shooting
{'Shooting': 2, 'Technical': 1}
(D.Mosher) Shooting
{'Personal': 1, 'Shooting': 1}
(D.Taylor) Offensive
{'Loose Ball': 1, 'Shooting': 5}
(D.Taylor) Loose Ball
{'Loose Ball': 1, 'Shooting': 5, 'Offensive': 1}
(D.Taylor) Offensive
{'Loose Ball': 2, 'Shooting': 5, 'Offensive': 1}
(C.Kirkland) Personal
{'Shooting': 3, 'Technical': 1}
(C.Kirkland) Shooting
{'Shooting': 3, 'Technical': 1, 'Personal': 1}
(D.Mosher) Personal
{'Personal': 1, 'Shooting': 2}
(D.Taylor) 

{'(D.Taylor)': {'Loose Ball': 2, 'Shooting': 8, 'Offensive': 4, 'Personal': 4},
 '(C.Kirkland)': {'Shooting': 10,
  'Technical': 1,
  'Personal': 3,
  'Loose Ball': 1},
 '(D.Mosher)': {'Personal': 3, 'Shooting': 7, 'Loose Ball': 1},
 '(S.Wall)': {'Personal': 4, 'Shooting': 6, 'Loose Ball': 1},
 '(A.Smith)': {'Shooting': 5, 'Offensive': 1, 'Personal': 1},
 '(J.Capers)': {'Shooting': 9, 'Personal': 8},
 '(G.Petraitis)': {'Shooting': 7,
  'Personal': 2,
  'Offensive': 1,
  'Loose Ball': 1},
 '(S.Mehta)': {'Shooting': 9, 'Personal': 5, 'Technical': 1},
 '(D.Collins)': {'Personal': 3, 'Shooting': 6, 'Offensive': 1},
 '(D.Scott)': {'Shooting': 1, 'Personal': 3, 'Personal Take': 1},
 '(M.Davis)': {'Offensive': 1, 'Personal': 2, 'Offensive Charge': 1},
 '(B.Taylor)': {'Personal': 3},
 '(M.Boland)': {'Personal': 7, 'Shooting': 4},
 '(S.Wright)': {'Shooting': 5, 'Personal': 3, 'Loose Ball': 4},
 '(K.Lane)': {'Shooting': 10,
  'Personal': 3,
  'Offensive Charge': 1,
  'Loose Ball': 1},
 '(S.Corbi